In [ ]:
#| default_exp models.MMM0XX

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import torch, re, inspect, pickle, os, torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
from typing import Optional, List, Tuple, Mapping, Any, Union
from transformers import (
    PretrainedConfig,
    BertLMHeadModel, 
    BatchEncoding, 
    BertPreTrainedModel, 
    BertModel, 
    RobertaForCausalLM, 
    DistilBertForMaskedLM,
    DistilBertModel,
    DistilBertPreTrainedModel,
)
from transformers.utils.generic import ModelOutput

from fastcore.meta import *

from xcai.losses import *
from xcai.core import store_attr

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
from xcai.block import *

## Setup

In [ ]:
block = XCBlock.from_cfg('/home/aiscuser/scratch/datasets', 'data_metas', tfm='xcnlg', 
                         tokenizer='distilbert-base-uncased', 
                         smp_features=[('lbl2data',1, 2), ('cat2data',1, 1), ('cat2lbl2data',2,1), 
                                       ('hlk2data', 1, 3)])

/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
data_dir = '/home/aiscuser/scratch/datasets'
pkl_dir = f'{data_dir}/processed'

In [ ]:
fname = f'{pkl_dir}/wikiseealso_data-metas_distilbert-base-uncased_xcnlg_radga.pkl'

In [ ]:
with open(fname, 'wb') as file: pickle.dump(block, file)

In [ ]:
with open(fname, 'rb') as file: block = pickle.load(file)

In [ ]:
batch = block.train.one_batch(5)

for i,batch in enumerate(block.train.dl):
    if i > 3: break

## Output

In [ ]:
#| export
@dataclass
class XCModelOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: Optional[torch.FloatTensor] = None
    lm_loss: Optional[torch.FloatTensor] = None
    dr_loss: Optional[torch.FloatTensor] = None
    data_repr: Optional[torch.FloatTensor] = None
    lbl2data_repr: Optional[torch.FloatTensor] = None
    data_hidden_states: Optional[Tuple[torch.FloatTensor, ...]] = None
    data_attentions: Optional[Tuple[torch.FloatTensor, ...]] = None
    data_cross_attentions: Optional[Tuple[torch.FloatTensor, ...]] = None
    lbl2data_hidden_states: Optional[Tuple[torch.FloatTensor, ...]] = None
    lbl2data_attentions: Optional[Tuple[torch.FloatTensor, ...]] = None
    lbl2data_cross_attentions: Optional[Tuple[torch.FloatTensor, ...]] = None
    

## Pooling

In [ ]:
#| export
class Pooling:

    @staticmethod
    def mean_pooling(data_embeds:torch.FloatTensor, data_attention_mask:torch.LongTensor):
        data_attention_mask = data_attention_mask.unsqueeze(2).expand(data_embeds.size()).float()
        return torch.sum(data_embeds * data_attention_mask, 1) / torch.clamp(data_attention_mask.sum(1), min=1e-9)


## BT0001

In [ ]:
#| export
class BT0001(BertLMHeadModel):

    def __init__(self, cfg):
        super().__init__(cfg)

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        data_token_type_ids:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_token_type_ids:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        data_o = self.bert(
            data_input_ids,
            data_attention_mask,
            data_token_type_ids,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        data_logits = self.cls(data_o[0])
        data_repr = data_o[0].mean(dim=1)
        
        if lbl2data_input_ids is not None and lbl2data_data2ptr is not None:
            lbl2data_o = self.bert(
                lbl2data_input_ids,
                lbl2data_attention_mask,
                lbl2data_token_type_ids,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict
            )
            lbl2data_logits = self.cls(lbl2data_o[0][lbl2data_data2ptr.cumsum(dim=0)-1])
            lbl2data_repr = lbl2data_o[0].mean(dim=1)
            
            return data_logits, lbl2data_input_ids, lbl2data_data2ptr, lbl2data_idx, lbl2data_logits, data_input_ids, data_repr, lbl2data_repr, data_o[0], data_attention_mask, lbl2data_o[0], lbl2data_attention_mask, kwargs

        return data_logits, lbl2data_input_ids, lbl2data_data2ptr, lbl2data_idx, kwargs
        

In [ ]:
m = BT0001.from_pretrained('bert-base-uncased')

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [ ]:
out = m(**batch)

In [ ]:
for o in out: print(o.shape)

torch.Size([20, 18, 30522])
torch.Size([37, 16])
torch.Size([20])
torch.Size([20, 768])
torch.Size([37, 768])


## BT0002

In [ ]:
#| export
class BT0002(BertLMHeadModel):
    use_generation,use_representation = True,False 

    def __init__(self,
                 config,
                 tn_targ:Optional[int]=None, 
                 ig_tok:Optional[int]=0,
                 *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.loss_fn = MultiCrossEntropy(tn_targ=tn_targ, ig_tok=ig_tok, reduce='mean')

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        data_token_type_ids:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_token_type_ids:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        data_o = self.bert(
            data_input_ids,
            data_attention_mask,
            data_token_type_ids,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        data_logits = self.cls(data_o[0])
        
        loss = None
        if lbl2data_input_ids is not None:
            loss = self.loss_fn(data_logits, lbl2data_input_ids, lbl2data_data2ptr, **kwargs)

        if not return_dict:
            o = (data_logits,) + data_o[2:]
            return ((loss,) + o) if loss is not None else o

        return XCModelOutput(
            loss=loss,
            logits=data_logits,
            data_hidden_states=data_o.hidden_states,
            data_attentions=data_o.attentions,
            data_cross_attentions=data_o.cross_attentions,
        )
        

### Example

In [ ]:
m = BT0002.from_pretrained('bert-base-uncased', tn_targ=10_000, ig_tok=0)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BT0002 were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['loss_fn.o']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
b = prepare_batch(m, batch)

In [ ]:
m, b = m.to('cuda'), b.to('cuda')

In [ ]:
o = m(**b)

In [ ]:
o.loss

tensor(13.6697, device='cuda:0', grad_fn=<SumBackward0>)

## BT0003

In [ ]:
#| export
class BT0003(BertPreTrainedModel):
    use_generation,use_representation = False,True
    
    def __init__(self,
                 config,
                 bsz:Optional[int]=None,
                 tn_targ:Optional[int]=None,
                 margin:Optional[float]=0.8,
                 tau:Optional[float]=0.1,
                 apply_softmax=False,
                 *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.bert = BertModel(config)
        self.loss_fn = MultiTriplet(bsz=bsz, tn_targ=tn_targ, margin=margin, tau=tau, apply_softmax=apply_softmax, reduce='mean')
        self.post_init()

    @delegates(BertModel.__call__)
    def get_repr(self, 
                 input_ids:Optional[torch.Tensor]=None, 
                 attention_mask:Optional[torch.Tensor]=None,
                 token_type_ids:Optional[torch.Tensor]=None,
                 **kwargs):
        o = self.bert(
            input_ids,
            attention_mask,
            token_type_ids,
            **kwargs
        )
        return o, Pooling.mean_pooling(o[0], attention_mask)

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        data_token_type_ids:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_token_type_ids:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        data_o, data_repr = self.get_repr(data_input_ids, 
                                          data_attention_mask, 
                                          data_token_type_ids, 
                                          output_attentions=output_attentions, 
                                          output_hidden_states=output_hidden_states,
                                          return_dict=return_dict)
        loss, lbl2data_repr = None, None
        if lbl2data_input_ids is not None:
            lbl2data_o, lbl2data_repr = self.get_repr(lbl2data_input_ids, 
                                                      lbl2data_attention_mask, 
                                                      lbl2data_token_type_ids, 
                                                      output_attentions=output_attentions, 
                                                      output_hidden_states=output_hidden_states,
                                                      return_dict=return_dict)
            loss = self.loss_fn(data_repr, lbl2data_repr, lbl2data_data2ptr, lbl2data_idx, **kwargs)

        if not return_dict:
            o = (data_repr, lbl2data_repr)
            return ((loss,) + o) if loss is not None else o

        return XCModelOutput(
            loss=loss,
            data_repr=data_repr,
            lbl2data_repr=lbl2data_repr,
        )
        

### Example

In [ ]:
m = BT0003.from_pretrained('bert-base-uncased', tn_targ=10_000)

Some weights of BT0003 were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['loss_fn.v']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
b = prepare_batch(m, batch)

In [ ]:
b.keys()

dict_keys(['lbl2data_idx', 'lbl2data_input_ids', 'lbl2data_token_type_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'data_input_ids', 'data_token_type_ids', 'data_attention_mask'])

In [ ]:
m, b = m.to('cuda'), b.to('cuda')

In [ ]:
o = m(**b)

/scratch/scai/phd/aiz218323/Projects/xcai/xcai/losses.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [ ]:
o.data_repr.shape

torch.Size([20, 768])

## BT0004

In [ ]:
#| export
class BT0004(BertLMHeadModel):
    use_generation,use_representation = True,True 

    def __init__(self,
                 config,
                 tau:Optional[float]=1.0,
                 tn_targ:Optional[int]=None, 
                 ig_tok:Optional[int]=0,
                 lw:Optional[int]=0.5,
                 *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.lw, self.dr_loss_fn = lw, SoupCon(tau=tau, reduce='mean')
        self.lm_loss_fn = MultiCrossEntropy(tn_targ=tn_targ, ig_tok=ig_tok, reduce='mean')
        
    @delegates(BertModel.__call__)
    def get_repr(self, 
                 input_ids:Optional[torch.Tensor]=None, 
                 attention_mask:Optional[torch.Tensor]=None,
                 token_type_ids:Optional[torch.Tensor]=None,
                 **kwargs):
        o = self.bert(
            input_ids,
            attention_mask,
            token_type_ids,
            **kwargs
        )
        return o, F.normalize(o[0].mean(dim=1), dim=1)

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        data_token_type_ids:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_token_type_ids:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        data_o, data_repr = self.get_repr(
            data_input_ids,
            data_attention_mask,
            data_token_type_ids,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        data_logits = self.cls(data_o[0])
        
        loss, lm_loss, dr_loss, lbl2data_repr = None, None, None, None
        if lbl2data_input_ids is not None:
            lbl2data_o, lbl2data_repr = self.get_repr(
                lbl2data_input_ids,
                lbl2data_attention_mask,
                lbl2data_token_type_ids,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )
            
            lm_loss = self.lm_loss_fn(data_logits, lbl2data_input_ids, lbl2data_data2ptr)
            dr_loss = self.dr_loss_fn(data_repr, lbl2data_repr, lbl2data_data2ptr, lbl2data_idx, **kwargs)
            loss = lm_loss + self.lw*dr_loss
            
        if not return_dict:
            o = (data_logits,data_repr,lbl2data_repr) + data_o[2:]
            return ((loss,lm_loss,dr_loss) + o) if loss is not None else o

        return XCModelOutput(
            loss=loss,
            lm_loss=lm_loss,
            dr_loss=dr_loss,
            logits=data_logits,
            data_repr=data_repr,
            lbl2data_repr=lbl2data_repr,
            data_hidden_states=data_o.hidden_states,
            data_attentions=data_o.attentions,
            data_cross_attentions=data_o.cross_attentions,
        )
        

### Example

In [ ]:
b = prepare_batch(m, batch)

In [ ]:
m = BT0004.from_pretrained('bert-base-uncased', lw=0.5, tn_targ=10_000, ig_tok=0)
m, b = m.to('cuda'), b.to('cuda')
o = m(**b)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BT0004 were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['dr_loss_fn.tau', 'lm_loss_fn.o']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## RT0005

In [ ]:
#| export
class RT0005(RobertaForCausalLM):
    use_generation,use_representation = True,False 

    def __init__(self,
                 config,
                 tn_targ:Optional[int]=None, 
                 ig_tok:Optional[int]=0,
                 *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.loss_fn = MultiCrossEntropy(tn_targ=tn_targ, ig_tok=ig_tok, reduce='mean')

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        data_o = self.roberta(
            data_input_ids,
            data_attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        data_logits = self.lm_head(data_o[0])
        
        loss = None
        if lbl2data_input_ids is not None:
            loss = self.loss_fn(data_logits, lbl2data_input_ids, lbl2data_data2ptr, **kwargs)

        if not return_dict:
            o = (data_logits,) + data_o[2:]
            return ((loss,) + o) if loss is not None else o

        return XCModelOutput(
            loss=loss,
            logits=data_logits,
            data_hidden_states=data_o.hidden_states,
            data_attentions=data_o.attentions,
            data_cross_attentions=data_o.cross_attentions,
        )
        

### Example

In [ ]:
m = RT0005.from_pretrained('roberta-base', tn_targ=10_000, ig_tok=0)

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of RT0005 were not initialized from the model checkpoint at roberta-base and are newly initialized: ['loss_fn.o']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
b = prepare_batch(m, batch)

In [ ]:
m, b = m.to('cuda'), b.to('cuda')

In [ ]:
o = m(**b)

In [ ]:
o.loss

tensor(17.3047, device='cuda:0', grad_fn=<SumBackward0>)

## BT0006

In [ ]:
#| export
class BT0006(BT0003):
    use_generation,use_representation = False,True

    @delegates(BT0003.__init__)
    def __init__(self, config, tau:Optional[int]=1.0, *args, **kwargs):
        super().__init__(config, tau=tau, *args, **kwargs)
        self.loss_fn = SoupCon(tau=tau, reduce='mean')
        

### Example

In [ ]:
m = BT0006.from_pretrained('bert-base-uncased', tau=1.0)

Some weights of BT0006 were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['loss_fn.tau']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
b = prepare_batch(m, batch)

In [ ]:
m, b = m.to('cuda'), b.to('cuda')

In [ ]:
o = m(**b)

In [ ]:
o.loss

tensor(16.9917, device='cuda:0', grad_fn=<DivBackward0>)

## DBT007

In [ ]:
#| export
class DBT007(DistilBertForMaskedLM):
    use_generation,use_representation = True,False 
    
    def __init__(self, 
                 config,
                 tn_targ:Optional[int]=None, 
                 ig_tok:Optional[int]=0,
                 vocab_weights:Optional[torch.Tensor]=None,
                 reduction:Optional[str]='mean',
                ):
        super().__init__(config)
        self.loss_fn = MultiCrossEntropy(tn_targ=tn_targ, ig_tok=ig_tok, vocab_weights=vocab_weights, reduce=reduction)

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        data_o = self.distilbert(
            input_ids=data_input_ids,
            attention_mask=data_attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        data_logits = self.vocab_transform(data_o[0])
        data_logits = self.activation(data_logits)
        data_logits = self.vocab_layer_norm(data_logits)
        data_logits = self.vocab_projector(data_logits)

        loss = None
        if lbl2data_input_ids is not None:
            loss = self.loss_fn(data_logits, lbl2data_input_ids, lbl2data_data2ptr, **kwargs)

        if not return_dict:
            o = (data_logits,) + data_o[2:]
            return ((loss,) + o) if loss is not None else o

        return XCModelOutput(
            loss=loss,
            logits=data_logits,
            data_hidden_states=data_o.hidden_states,
            data_attentions=data_o.attentions,
        )


In [ ]:
batch.keys()

dict_keys(['plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_identifier', 'cat2data_input_text', 'cat2data_input_ids', 'cat2data_attention_mask', 'cat2data_data2ptr', 'pcat2lbl2data_idx', 'pcat2lbl2data_data2ptr', 'cat2lbl2data_idx', 'cat2lbl2data_identifier', 'cat2lbl2data_input_text', 'cat2lbl2data_input_ids', 'cat2lbl2data_attention_mask', 'cat2lbl2data_data2ptr', 'phlk2data_idx', 'phlk2data_data2ptr', 'hlk2data_idx', 'hlk2data_identifier', 'hlk2data_input_text', 'hlk2data_input_ids', 'hlk2data_attention_mask', 'hlk2data_data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'hlk2lbl2data_idx', 'hlk2lbl2data_identifier', 'hlk2lbl2data_input_text', 'hlk2lbl2data_input_ids', 'hlk2lbl2data_attention_mask', 'hlk2lbl2data_data2ptr', 'hlk2lbl2data_plbl2data2ptr'])

### Example

In [ ]:
m = DBT007.from_pretrained('distilbert-base-uncased', ig_tok=0)

In [ ]:
o = m(**batch)

In [ ]:
o.loss

tensor(15.4827, grad_fn=<SumBackward0>)

## DBT008

In [ ]:
#| export
class DBT008(DistilBertPreTrainedModel):
    use_generation,use_representation = False,True
    
    def __init__(self,
                 config,
                 bsz:Optional[int]=None,
                 tn_targ:Optional[int]=None,
                 margin:Optional[float]=0.8,
                 tau:Optional[float]=0.1,
                 apply_softmax:Optional[bool]=False,
                 n_negatives:Optional[int]=5,
                 *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.distilbert = DistilBertModel(config)
        self.dr_transform = nn.Linear(config.dim, config.dim)
        self.dr_layer_norm = nn.LayerNorm(config.dim, eps=1e-12)
        self.dr_projector = nn.Linear(config.dim, config.dim)
        
        self.loss_fn = MultiTriplet(bsz=bsz, tn_targ=tn_targ, margin=margin, n_negatives=n_negatives, tau=tau, 
                                    apply_softmax=apply_softmax, reduce='mean')
        self.post_init()
        
    def init_dr_head(self):
        self.dr_transform.weight.data = torch.eye(self.dr_transform.out_features, self.dr_transform.in_features, 
                                                  dtype=self.dr_transform.weight.dtype)
        self.dr_projector.weight.data = torch.eye(self.dr_projector.out_features, self.dr_projector.in_features, 
                                                  dtype=self.dr_projector.weight.dtype)
        
    @delegates(BertModel.__call__)
    def get_repr(self, 
                 input_ids:Optional[torch.Tensor]=None, 
                 attention_mask:Optional[torch.Tensor]=None,
                 **kwargs):
        o = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        rep = self.dr_transform(o[0])
        rep = self.dr_layer_norm(rep)
        rep = self.dr_projector(rep)
        return o, Pooling.mean_pooling(rep, attention_mask)

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        data_o, data_repr = self.get_repr(data_input_ids, 
                                          data_attention_mask, 
                                          output_attentions=output_attentions, 
                                          output_hidden_states=output_hidden_states,
                                          return_dict=return_dict)
        loss, lbl2data_repr = None, None
        if lbl2data_input_ids is not None:
            lbl2data_o, lbl2data_repr = self.get_repr(lbl2data_input_ids, 
                                                      lbl2data_attention_mask,  
                                                      output_attentions=output_attentions, 
                                                      output_hidden_states=output_hidden_states,
                                                      return_dict=return_dict)
            loss = self.loss_fn(data_repr, lbl2data_repr, lbl2data_data2ptr, lbl2data_idx, 
                                plbl2data_data2ptr, plbl2data_idx, **kwargs)

        if not return_dict:
            o = (data_repr, lbl2data_repr)
            return ((loss,) + o) if loss is not None else o

        return XCModelOutput(
            loss=loss,
            data_repr=data_repr,
            lbl2data_repr=lbl2data_repr,
        )
        

### Example

In [ ]:
m = DBT008.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=1024, margin=0.3, tau=0.1, 
                           n_negatives=5, apply_softmax=True)

Some weights of DBT008 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
m.init_dr_head()

In [ ]:
o = m(**batch)

In [ ]:
o.loss, o.data_repr.shape, o.lbl2data_repr.shape

(tensor(0.6046, grad_fn=<DivBackward0>),
 torch.Size([5, 768]),
 torch.Size([6, 768]))

In [ ]:
o.loss

tensor(0.6046, grad_fn=<DivBackward0>)

## DBT009

In [ ]:
#| export
class DBT009(DBT008):

    @delegates(DBT008.__init__)
    def __init__(self, config, *args, **kwargs):
        super().__init__(config, *args, **kwargs)

    def get_repr(self, 
                 input_ids:Optional[torch.Tensor]=None, 
                 attention_mask:Optional[torch.Tensor]=None,
                 **kwargs):
        o,rep = super().get_repr(input_ids, attention_mask, **kwargs)
        return o,F.normalize(rep, dim=1)
        

### Example

In [ ]:
m = DBT009.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=1024, margin=0.3, tau=0.1, 
                           n_negatives=5, apply_softmax=True)

Some weights of DBT009 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
m.init_dr_head()

In [ ]:
o = m(**batch)

In [ ]:
o.loss

tensor(0.0319, grad_fn=<DivBackward0>)

In [ ]:
m.loss_fn.bsz, m.loss_fn.margin, m.loss_fn.tau, m.loss_fn.n_negatives

(1024, 0.3, 0.1, 5)

## DBT010

In [ ]:
#| export
class DBT010(DBT009):

    @delegates(DBT009.__init__)
    def __init__(self, config, tau:Optional[int]=1.0, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.loss_fn = SoupCon(tau=tau, reduce='mean')
        

### Example

In [ ]:
m = DBT010.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', tau=0.2)

Some weights of DBT010 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['loss_fn.tau']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
o = m(**batch)

In [ ]:
m.loss_fn.tau

Parameter containing:
tensor(0.2000, requires_grad=True)

In [ ]:
o.loss

tensor(1.3032, grad_fn=<MeanBackward0>)

## DBT011

In [ ]:
#| export
class DBT011(DBT008):

    @delegates(DBT008.__init__)
    def __init__(self, config, 
                 margin:Optional[float]=0.3, 
                 tau:Optional[float]=0.1, 
                 apply_softmax:Optional[bool]=True,
                 n_negatives:Optional[int]=5,
                 *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.lfn, self.loss_fn = Triplet(margin=margin, tau=tau, n_negatives=n_negatives, apply_softmax=apply_softmax, reduce='mean'), None
        
    def loss_fn(self,
                data_repr:torch.Tensor, 
                lbl2data_repr:torch.Tensor, 
                lbl2data_data2ptr:torch.Tensor, 
                lbl2data_idx:torch.Tensor, 
                plbl2data_data2ptr:torch.Tensor, 
                plbl2data_idx:torch.Tensor, 
                **kwargs):
        return self.lfn(data_repr, lbl2data_repr, lbl2data_idx, plbl2data_data2ptr, plbl2data_idx)
        

### Example

In [ ]:
m = DBT011.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', margin=0.4, tau=0.7, 
                           apply_softmax=True, n_negatives=5)
b = prepare_batch(m, batch, m_args=['plbl2data_idx', 'plbl2data_data2ptr'])

In [ ]:
o = m(**b)

/home/aiscuser/scratch/Projects/xcai/xcai/losses.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [ ]:
o.loss

tensor(0.8743, grad_fn=<MeanBackward0>)

## DBT012

In [ ]:
#| export
class DBT012(DBT009):

    @delegates(DBT009.__init__)
    def __init__(self, 
                 config, 
                 margin:Optional[float]=0.3, 
                 tau:Optional[float]=0.1, 
                 apply_softmax:Optional[bool]=True,
                 n_negatives:Optional[int]=5,
                 *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.lfn, self.loss_fn = Triplet(margin=margin, tau=tau, n_negatives=n_negatives, apply_softmax=apply_softmax, reduce='mean'), None
        
    def loss_fn(self,
                data_repr:torch.Tensor, 
                lbl2data_repr:torch.Tensor, 
                lbl2data_data2ptr:torch.Tensor, 
                lbl2data_idx:torch.Tensor, 
                plbl2data_data2ptr:torch.Tensor, 
                plbl2data_idx:torch.Tensor, 
                **kwargs):
        return self.lfn(data_repr, lbl2data_repr, lbl2data_idx, plbl2data_data2ptr, plbl2data_idx)


### Example

In [ ]:
m = DBT012.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', margin=0.4, tau=0.7, 
                           apply_softmax=True, n_negatives=5)
b = prepare_batch(m, batch, m_args=['plbl2data_idx', 'plbl2data_data2ptr'])

In [ ]:
o = m(**b)

In [ ]:
o.loss

tensor(0.0652, grad_fn=<MeanBackward0>)

In [ ]:
m.lfn.margin, m.lfn.tau, m.lfn.apply_softmax, m.lfn.n_negatives

(0.4, 0.7, True, 5)

## DBT013

In [ ]:
#| export
class DBT013(DistilBertForMaskedLM):
    use_generation,use_representation = True,True 
    
    def __init__(
        self, config,
        tn_targ:Optional[int]=None, 
        ig_tok:Optional[int]=0,
        bsz:Optional[int]=None,
        margin:Optional[int]=0.3,
        n_negatives:Optional[int]=5,
        tau:Optional[float]=0.1,
        apply_softmax:Optional[bool]=True,
        lw:Optional[float]=0.8,
    ):
        super().__init__(config)
        self.dr_transform = nn.Linear(config.dim, config.dim)
        self.dr_layer_norm = nn.LayerNorm(config.dim, eps=1e-12)
        self.dr_projector = nn.Linear(config.dim, config.dim)
        
        self.lw = lw
        self.gen_lfn = MultiCrossEntropy(tn_targ=tn_targ, ig_tok=ig_tok, reduce='mean')
        self.rep_lfn = MultiTriplet(bsz=bsz, tn_targ=tn_targ, margin=margin, n_negatives=n_negatives, tau=tau,
                                    apply_softmax=apply_softmax, reduce='mean')
        
    def init_dr_head(self):
        self.dr_transform.weight.data = torch.eye(self.dr_transform.out_features, self.dr_transform.in_features, 
                                                  dtype=self.dr_transform.weight.dtype)
        self.dr_projector.weight.data = torch.eye(self.dr_projector.out_features, self.dr_projector.in_features, 
                                                  dtype=self.dr_projector.weight.dtype)
        
    def get_representation(self, input_ids:Optional[torch.Tensor]=None, attention_mask:Optional[torch.Tensor]=None, **kwargs):
        o = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        rep = self.dr_transform(o[0])
        rep = self.dr_layer_norm(rep)
        rep = self.dr_projector(rep)
        return o, F.normalize(Pooling.mean_pooling(rep, attention_mask), dim=1)
    
    def get_generation(self, input_ids:Optional[torch.Tensor]=None, attention_mask:Optional[torch.Tensor]=None, **kwargs):
        o = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        logits = self.vocab_transform(o[0])
        logits = self.activation(logits)
        logits = self.vocab_layer_norm(logits)
        return o, self.vocab_projector(logits)
        
    def get_genrep(self, input_ids:Optional[torch.Tensor]=None, attention_mask:Optional[torch.Tensor]=None, **kwargs):    
        o = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        rep = self.dr_transform(o[0])
        rep = self.dr_layer_norm(rep)
        rep = self.dr_projector(rep)
        rep = F.normalize(Pooling.mean_pooling(rep, attention_mask), dim=1)
        
        logits = self.vocab_transform(o[0])
        logits = self.activation(logits)
        logits = self.vocab_layer_norm(logits)
        logits = self.vocab_projector(logits)
        return o,logits,rep

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        data_o, data_logits, data_repr = self.get_genrep(data_input_ids, data_attention_mask)
        
        loss = lm_loss = dr_loss = lbl2data_repr = None
        if lbl2data_input_ids is not None:
            lbl2data_o, lbl2data_repr = self.get_representation(lbl2data_input_ids, lbl2data_attention_mask)
            
            lm_loss = self.gen_lfn(data_logits, lbl2data_input_ids, lbl2data_data2ptr, **kwargs)
            dr_loss = self.rep_lfn(data_repr, lbl2data_repr, lbl2data_data2ptr, lbl2data_idx, 
                                   plbl2data_data2ptr, plbl2data_idx, **kwargs)
            loss = dr_loss + self.lw*lm_loss
            
        if not return_dict:
            o = (data_logits,data_repr,lbl2data_repr) + data_o[2:]
            return ((loss,lm_loss,dr_loss) + o) if loss is not None else o
        
        return XCModelOutput(
            loss=loss,
            lm_loss=lm_loss,
            dr_loss=dr_loss,
            logits=data_logits,
            data_repr=data_repr,
            lbl2data_repr=lbl2data_repr,
            data_hidden_states=data_o.hidden_states,
            data_attentions=data_o.attentions,
        )
    

### Example

In [ ]:
m = DBT013.from_pretrained('distilbert-base-uncased', tn_targ=10_000, ig_tok=0,
                           margin=0.4, tau=0.7, apply_softmax=True, n_negatives=5, lw=0.8)
b = prepare_batch(m, batch, m_args='lbl2data_idx')

Some weights of DBT013 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for k,v in b.items(): print(k, ': ', v.shape)

plbl2data_idx :  torch.Size([8])
plbl2data_data2ptr :  torch.Size([5])
lbl2data_idx :  torch.Size([6])
lbl2data_input_ids :  torch.Size([6, 13])
lbl2data_attention_mask :  torch.Size([6, 13])
lbl2data_data2ptr :  torch.Size([5])
data_input_ids :  torch.Size([5, 10])
data_attention_mask :  torch.Size([5, 10])


In [ ]:
o = m(**b)

In [ ]:
o.loss

tensor(9.3832, grad_fn=<AddBackward0>)

## DBT014

In [ ]:
#| export
class DBT014(DBT013):
    
    @delegates(DBT013.__init__)
    def __init__(self, config, m_lw:Optional[Union[float,List]]=0.2, meta_prefix:Optional[str]=None, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.m_lw, self.meta_prefix = m_lw, meta_prefix
        
    def _get_meta_inputs(self, **kwargs):
        inputs = {}
        for t in [o for o in kwargs if self.meta_prefix is not None and re.match(f'^[p]?{self.meta_prefix}.*', o)]:
            p,q = t.split('_', maxsplit=1)
            if t[0] == 'p': inputs.setdefault(p[1:], {})[f'p{q}'] = kwargs[t]
            else: inputs.setdefault(p, {})[q] = kwargs[t]
        return inputs

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        data_o, data_logits, data_repr = self.get_genrep(data_input_ids, data_attention_mask)
        
        loss = lm_loss = dr_loss = lbl2data_repr = None
        if lbl2data_input_ids is not None:
            lbl2data_o, lbl2data_repr = self.get_representation(lbl2data_input_ids, lbl2data_attention_mask)
            lm_loss = self.gen_lfn(data_logits, lbl2data_input_ids, lbl2data_data2ptr, **kwargs)
            dr_loss = self.rep_lfn(data_repr, lbl2data_repr, lbl2data_data2ptr, lbl2data_idx, 
                                   plbl2data_data2ptr, plbl2data_idx, **kwargs)
            loss = dr_loss + self.lw*lm_loss
            
            meta_inputs = self._get_meta_inputs(**kwargs)
            if isinstance(self.m_lw, float):
                meta_lw = self.m_lw/len(meta_inputs) if len(meta_inputs) else None
                meta_lw = [meta_lw]*len(meta_inputs)
            else:
                if len(self.m_lw) != len(meta_inputs): raise ValueError(f'length of `m_lw` should be equal to number of metadata.')
                meta_lw = self.m_lw
                
            for m,m_lw in zip(meta_inputs.values(),meta_lw):
                valid_idx = torch.where(m['data2ptr'])[0]
                if len(valid_idx) > 0:
                    o, rep = self.get_representation(m['input_ids'], m['attention_mask'])
                    m_lml = self.gen_lfn(data_logits[valid_idx], m['input_ids'], m['data2ptr'][valid_idx], **kwargs)
                    m_drl = self.rep_lfn(data_repr[valid_idx], rep, m['data2ptr'][valid_idx], m['idx'], 
                                         m['pdata2ptr'][valid_idx], m['pidx'], **kwargs)
                    loss += m_lw * (m_drl + self.lw*m_lml)
            
        if not return_dict:
            o = (data_logits,data_repr,lbl2data_repr) + data_o[2:]
            return ((loss,lm_loss,dr_loss) + o) if loss is not None else o
        
        return XCModelOutput(
            loss=loss,
            lm_loss=lm_loss,
            dr_loss=dr_loss,
            logits=data_logits,
            data_repr=data_repr,
            lbl2data_repr=lbl2data_repr,
            data_hidden_states=data_o.hidden_states,
            data_attentions=data_o.attentions,
        )
    

### Example

In [ ]:
m = DBT014.from_pretrained('distilbert-base-uncased', tn_targ=10_000, ig_tok=0, margin=0.4, tau=0.7, 
                           apply_softmax=True, n_negatives=5, lw=0.8, m_lw=[0.2], meta_prefix='hlk')

b = prepare_batch(m, batch, m_args=['hlk2data_input_ids', 'hlk2data_attention_mask', 'hlk2data_data2ptr', 
                                    'hlk2data_idx', 'phlk2data_data2ptr', 'phlk2data_idx'])

Some weights of DBT014 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
o = m(**b)

In [ ]:
o.loss

tensor(11.9105, grad_fn=<AddBackward0>)

## DBT015

In [ ]:
#| export
class DBT015(DBT013):
    
    @delegates(DBT013.__init__)
    def __init__(self, config, m_lw:Optional[float]=0.2, meta_prefix:Optional[str]=None, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.mdr_lfn = Triplet(margin=kwargs.get('margin'), tau=kwargs.get('tau'), n_negatives=kwargs.get('n_negatives'), 
                               apply_softmax=kwargs.get('apply_softmax'), reduce='mean')
        self.m_lw, self.meta_prefix = m_lw, meta_prefix 
        
    def _get_meta_inputs(self, **kwargs):
        inputs = {}
        for t in [o for o in kwargs if self.meta_prefix is not None and re.match(f'^[p]?{self.meta_prefix}.*', o)]:
            p,q = t.split('_', maxsplit=1)
            if t[0] == 'p': inputs.setdefault(p[1:], {})[f'p{q}'] = kwargs[t]
            else: inputs.setdefault(p, {})[q] = kwargs[t]
        return inputs

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        data_o, data_logits, data_repr = self.get_genrep(data_input_ids, data_attention_mask)
        
        loss = lm_loss = dr_loss = lbl2data_repr = None
        if lbl2data_input_ids is not None:
            lbl2data_o, lbl2data_repr = self.get_representation(lbl2data_input_ids, lbl2data_attention_mask)
            lm_loss = self.gen_lfn(data_logits, lbl2data_input_ids, lbl2data_data2ptr, **kwargs)
            dr_loss = self.rep_lfn(data_repr, lbl2data_repr, lbl2data_data2ptr, lbl2data_idx, **kwargs)
            loss = (lm_loss + self.lw*dr_loss) * (1-self.m_lw)
            
            meta_inputs = self._get_meta_inputs(**kwargs)
            m_lw = self.m_lw/len(meta_inputs)
            for m in meta_inputs.values():
                valid_idx = torch.where(m['data2ptr'])[0]
                if len(valid_idx) > 0:
                    o, rep = self.get_representation(m['input_ids'], m['attention_mask'])
                    m_lml = self.gen_lfn(data_logits[valid_idx], m['input_ids'], m['data2ptr'][valid_idx], **kwargs)
                    m_drl = self.mdr_lfn(data_repr[valid_idx], rep, m['idx'], m['pdata2ptr'][valid_idx], 
                                         m['pidx'], **kwargs)
                    loss += m_lw * (m_lml + self.lw*m_drl)
            
        if not return_dict:
            o = (data_logits,data_repr,lbl2data_repr) + data_o[2:]
            return ((loss,lm_loss,dr_loss) + o) if loss is not None else o
        
        return XCModelOutput(
            loss=loss,
            lm_loss=lm_loss,
            dr_loss=dr_loss,
            logits=data_logits,
            data_repr=data_repr,
            lbl2data_repr=lbl2data_repr,
            data_hidden_states=data_o.hidden_states,
            data_attentions=data_o.attentions,
        )
    

### Example

In [ ]:
m = DBT015.from_pretrained('distilbert-base-uncased', tn_targ=10_000, ig_tok=0, margin=0.4, tau=0.7, 
                           apply_softmax=True, n_negatives=5, lw=100.0, m_lw=0.2, meta_prefix='hlk')

b = prepare_batch(m, batch, m_args=['hlk2lbl2data_input_ids', 'hlk2lbl2data_attention_mask', 'hlk2lbl2data_data2ptr', 
                                    'hlk2lbl2data_idx', 'phlk2lbl2data_idx', 'phlk2lbl2data_data2ptr', 
                                    'hlk2data_input_ids', 'hlk2data_attention_mask', 'hlk2data_data2ptr', 
                                    'hlk2data_idx', 'phlk2data_idx', 'phlk2data_data2ptr', 
                                    'lbl2data_idx'])

Some weights of DBT015 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight', 'gen_lfn.o', 'rep_lfn.v']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch.keys()

dict_keys(['hlk2data_idx', 'phlk2data_idx', 'phlk2data_data2ptr', 'hlk2data_identifier', 'hlk2data_input_text', 'hlk2data_input_ids', 'hlk2data_attention_mask', 'hlk2data_data2ptr', 'hlk2lbl2data_idx', 'phlk2lbl2data_idx', 'phlk2lbl2data_data2ptr', 'hlk2lbl2data_identifier', 'hlk2lbl2data_input_text', 'hlk2lbl2data_input_ids', 'hlk2lbl2data_attention_mask', 'hlk2lbl2data_data2ptr', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'lbl2data_idx', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr'])

In [ ]:
o = m(**b)

In [ ]:
o.loss

tensor(22.0921, grad_fn=<AddBackward0>)

## DBT016

In [ ]:
#| export
class DBT016(DBT013):

    @delegates(DBT013.__init__)
    def __init__(self, config, *args, **kwargs):
        super().__init__(config, *args, **kwargs)

    def get_generation(self, input_ids:Optional[torch.Tensor]=None, attention_mask:Optional[torch.Tensor]=None, **kwargs):
        o = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        logits = o[0].detach()
        logits = self.vocab_transform(o[0])
        logits = self.activation(logits)
        logits = self.vocab_layer_norm(logits)
        return o, self.vocab_projector(logits)
        
    def get_genrep(self, input_ids:Optional[torch.Tensor]=None, attention_mask:Optional[torch.Tensor]=None, **kwargs):    
        o = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        rep = self.dr_transform(o[0])
        rep = self.dr_layer_norm(rep)
        rep = self.dr_projector(rep)
        rep = F.normalize(Pooling.mean_pooling(rep, attention_mask), dim=1)
        
        logits = o[0].detach()
        logits = self.vocab_transform(o[0])
        logits = self.activation(logits)
        logits = self.vocab_layer_norm(logits)
        logits = self.vocab_projector(logits)
        return o,logits,rep
    

### Example

In [ ]:
m = DBT016.from_pretrained('distilbert-base-uncased', tn_targ=10_000, ig_tok=0,
                           margin=0.4, tau=0.7, apply_softmax=True, n_negatives=5, lw=0.8)
b = prepare_batch(m, batch, m_args='lbl2data_idx')

/opt/conda/envs/ptca/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DBT016 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight', 'gen_lfn.o', 'rep_lfn.v']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
o = m(**b)

/home/aiscuser/scratch/Projects/xcai/xcai/losses.py:21: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [ ]:
o.loss

tensor(15.7218, grad_fn=<AddBackward0>)

## DBT017

In [ ]:
#| export
class DBT017(DBT013):
    
    @delegates(DBT013.__init__)
    def __init__(self, config, m_lw:Optional[Union[float,List]]=0.2, meta_prefix:Optional[str]=None, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.m_lw, self.meta_prefix = m_lw, meta_prefix
        
    def _get_meta_inputs(self, **kwargs):
        inputs = {}
        for t in [o for o in kwargs if self.meta_prefix is not None and re.match(f'^[p]?{self.meta_prefix}.*', o)]:
            p,q = t.split('_', maxsplit=1)
            if t[0] == 'p': inputs.setdefault(p[1:], {})[f'p{q}'] = kwargs[t]
            else: inputs.setdefault(p, {})[q] = kwargs[t]
        return inputs

    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        data_o, data_logits, data_repr = self.get_genrep(data_input_ids, data_attention_mask)
        
        loss = lm_loss = dr_loss = lbl2data_repr = None
        if lbl2data_input_ids is not None:
            lbl2data_o, lbl2data_logits, lbl2data_repr = self.get_genrep(lbl2data_input_ids, lbl2data_attention_mask)
            
            lm_loss = self.gen_lfn(data_logits, lbl2data_input_ids, lbl2data_data2ptr, **kwargs)
            dr_loss = self.rep_lfn(data_repr, lbl2data_repr, lbl2data_data2ptr, lbl2data_idx, 
                                   plbl2data_data2ptr, plbl2data_idx, **kwargs)
            loss = dr_loss + self.lw*lm_loss
            
            meta_inputs = self._get_meta_inputs(**kwargs)
            if isinstance(self.m_lw, float):
                meta_lw = self.m_lw/len(meta_inputs) if len(meta_inputs) else None
                meta_lw = [meta_lw]*len(meta_inputs)
            else:
                if len(self.m_lw) != len(meta_inputs): raise ValueError(f'length of `m_lw` should be equal to number of metadata.')
                meta_lw = self.m_lw
                
            for m,m_lw in zip(meta_inputs.values(), meta_lw):
                if 'lbl2data2ptr' in m:
                    valid_idx = torch.where(m['lbl2data2ptr'])[0]
                    if len(valid_idx) > 0:
                        o, rep = self.get_representation(m['input_ids'], m['attention_mask'])
                        m_lml = self.gen_lfn(lbl2data_logits[valid_idx], m['input_ids'], m['lbl2data2ptr'][valid_idx], **kwargs)
                        m_drl = self.rep_lfn(lbl2data_repr[valid_idx], rep, m['lbl2data2ptr'][valid_idx], m['idx'], 
                                             m['plbl2data2ptr'][valid_idx], m['pidx'], **kwargs)
                        loss += m_lw * (m_drl + self.lw* m_lml)
                        
                elif 'data2ptr' in m:
                    valid_idx = torch.where(m['data2ptr'])[0]
                    if len(valid_idx) > 0:
                        o, rep = self.get_representation(m['input_ids'], m['attention_mask'])
                        m_lml = self.gen_lfn(data_logits[valid_idx], m['input_ids'], m['data2ptr'][valid_idx], **kwargs)
                        m_drl = self.rep_lfn(data_repr[valid_idx], rep, m['data2ptr'][valid_idx], m['idx'], 
                                             m['pdata2ptr'][valid_idx], m['pidx'], **kwargs)
                        loss += m_lw * (m_drl + self.lw*m_lml) 
                        
                else: raise ValueError('Invalid metadata input arguments.')
            
        if not return_dict:
            o = (data_logits,data_repr,lbl2data_repr) + data_o[2:]
            return ((loss,lm_loss,dr_loss) + o) if loss is not None else o
        
        return XCModelOutput(
            loss=loss,
            lm_loss=lm_loss,
            dr_loss=dr_loss,
            logits=data_logits,
            data_repr=data_repr,
            lbl2data_repr=lbl2data_repr,
            data_hidden_states=data_o.hidden_states,
            data_attentions=data_o.attentions,
        )
    

### Example

In [ ]:
m = DBT017.from_pretrained('distilbert-base-uncased', tn_targ=10_000, ig_tok=0, margin=0.4, tau=0.7, 
                           apply_softmax=True, n_negatives=5, lw=0.8, m_lw=[0.1], meta_prefix='hlk')

b = prepare_batch(m, batch, m_args=['phlk2data_idx', 'phlk2data_data2ptr', 'hlk2data_idx', 'hlk2data_input_ids', 
                                    'hlk2data_attention_mask', 'hlk2data_data2ptr'])

Some weights of DBT017 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
o = m(**b)

In [ ]:
o.loss

tensor(10.6465, grad_fn=<AddBackward0>)

In [ ]:
b.keys()

dict_keys(['plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'phlk2data_idx', 'phlk2data_data2ptr', 'hlk2data_idx', 'hlk2data_input_ids', 'hlk2data_attention_mask', 'hlk2data_data2ptr', 'data_input_ids', 'data_attention_mask'])

## Fuser

In [ ]:
#| export
import math

In [ ]:
#| export
class Fuser(nn.Module):
    
    def __init__(self, config: PretrainedConfig):
        super().__init__()
        self.config, self.n_h, self.dim = config, config.n_heads, config.dim
        self.dropout = nn.Dropout(p=config.attention_dropout)

        if self.dim % self.n_h != 0:
            raise ValueError(f"self.n_heads: {self.n_h} must divide self.dim: {self.dim} evenly.")
            
        self.q = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.k = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.v = nn.Linear(in_features=config.dim, out_features=config.dim)
        self.o = nn.Linear(in_features=config.dim, out_features=config.dim)

    def forward(
        self, 
        data: torch.Tensor,
        data_mask: torch.Tensor,
        meta: torch.Tensor, 
        meta_mask: torch.Tensor,
        output_attentions:Optional[bool] = False,
    ):
        q, k, v, q_m, k_m = data, meta, meta, data_mask, meta_mask
        
        bs, q_len, dim = q.size()
        k_len = k.size(1)

        h_dim = self.dim//self.n_h

        def shape(x: torch.Tensor): return x.view(bs, -1, self.n_h, h_dim).transpose(1, 2)

        def unshape(x: torch.Tensor): return x.transpose(1, 2).contiguous().view(bs, -1, self.n_h * h_dim)

        q = shape(self.q(q))  # (bs, n_h, q_len, h_dim)
        k = shape(self.k(k))  # (bs, n_h, k_len, h_dim)
        v = shape(self.v(v))  # (bs, n_h, k_len, h_dim)

        q = q / math.sqrt(h_dim)  # (bs, n_h, q_len, h_dim)
        sc = torch.matmul(q, k.transpose(2, 3))  # (bs, n_h, q_len, k_len)
        
        q_m, k_m = q_m.view(bs, 1, -1, 1).to(q.dtype), k_m.view(bs, 1, 1, -1).to(q.dtype)
        mask = torch.matmul(q_m, k_m).expand_as(sc)  # (bs, n_h, q_len, k_len)
        
        sc = sc.masked_fill(mask.bool(), torch.tensor(torch.finfo(sc.dtype).min))  # (bs, n_h, q_len, k_len)

        w = nn.functional.softmax(sc, dim=-1)  # (bs, n_h, q_len, k_len)
        w = self.dropout(w)  # (bs, n_h, q_len, k_len)

        o = self.o(unshape(torch.matmul(w, v))) # (bs, q_len, dim)
        
        if output_attentions: return (o, w)
        else: return (o,)
        

### Example

In [ ]:
from transformers import AutoConfig

In [ ]:
config = AutoConfig.from_pretrained('distilbert-base-uncased')

/opt/conda/envs/ptca/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
fuser = Fuser(config)

In [ ]:
bsz, data_seq_len, meta_seq_len, dim, dtype = 10, 14, 17, config.dim, torch.float32
data, meta = torch.randn(bsz, data_seq_len, dim, dtype=dtype), torch.randn(bsz, meta_seq_len, dim, dtype=dtype)
data_mask = torch.randint(0, 2, size=(bsz,data_seq_len), dtype=dtype)
meta_mask = torch.randint(0, 2, size=(bsz,meta_seq_len), dtype=dtype)

In [ ]:
o = fuser(data, data_mask, meta, meta_mask)

In [ ]:
o[0].shape

torch.Size([10, 14, 768])

## DBT018

In [ ]:
#| export
from fastcore.utils import *

In [ ]:
#| export
class DBT018(DBT013):
    
    @delegates(DBT013.__init__)
    def __init__(self, config, aug_meta_prefix:Optional[List]=None, tn_meta:Optional[int]=None, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        store_attr('aug_meta_prefix')
        self.fuser = Fuser(config)
        self.ln = nn.LayerNorm(config.dim, eps=1e-12)
        self.o = torch.ones(tn_meta, dtype=torch.long, device=self.device) if tn_meta is not None else None
        
    def _get_meta_inputs(self, meta_prefix:Optional[str], **kwargs):
        inputs = {}
        for t in [o for o in kwargs if meta_prefix is not None and re.match(f'^[p]?{meta_prefix}.*', o)]:
            p,q = t.split('_', maxsplit=1)
            if t[0] == 'p': inputs.setdefault(p[1:], {})[f'p{q}'] = kwargs[t]
            else: inputs.setdefault(p, {})[q] = kwargs[t]
        return inputs
    
    def get_output(
        self, 
        input_ids:Optional[torch.Tensor]=None, 
        attention_mask:Optional[torch.Tensor]=None, 
        **kwargs
    ):
        o = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        return o
    
    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        data_o, data_logits, data_repr = self.get_meta_fused_genrep(data_input_ids, data_attention_mask, **kwargs)
        
        loss = lm_loss = dr_loss = lbl2data_repr = None
        if lbl2data_input_ids is not None:
            lbl2data_o, lbl2data_repr = self.get_representation(lbl2data_input_ids, lbl2data_attention_mask)
            lm_loss = self.gen_lfn(data_logits, lbl2data_input_ids, lbl2data_data2ptr, **kwargs)
            dr_loss = self.rep_lfn(data_repr, lbl2data_repr, lbl2data_data2ptr, lbl2data_idx, 
                                   plbl2data_data2ptr, plbl2data_idx, **kwargs)
            loss = dr_loss + self.lw*lm_loss
            
        if not return_dict:
            o = (data_logits,data_repr,lbl2data_repr) + data_o[2:]
            return ((loss,lm_loss,dr_loss) + o) if loss is not None else o
        
        return XCModelOutput(
            loss=loss,
            lm_loss=lm_loss,
            dr_loss=dr_loss,
            logits=data_logits,
            data_repr=data_repr,
            lbl2data_repr=lbl2data_repr,
            data_hidden_states=data_o[0],
        )
    
    def resize_meta(self, meta:torch.Tensor, mask:torch.Tensor, n_data2meta:torch.Tensor):
        bsz, dim, tn_data2meta = n_data2meta.shape[0], meta.shape[-1], meta.shape[0]
        self.o = self.o.to(meta.device)
        o = (
            torch.ones(tn_data2meta, dtype=torch.long, device=meta.device) 
            if self.o is None or len(self.o) < tn_data2meta else self.o[:tn_data2meta]
        )

        max_n_data2meta = n_data2meta.max()
        xn_data2meta = max_n_data2meta-n_data2meta+1

        data2meta_ptr = n_data2meta.cumsum(dim=0)-1
        r_data2meta = o.scatter(0, data2meta_ptr, xn_data2meta)

        xmeta,xmask = meta.repeat_interleave(r_data2meta, dim=0),mask.repeat_interleave(r_data2meta, dim=0)
        m = o.scatter(0, data2meta_ptr, 0).repeat_interleave(r_data2meta, dim=0).view(bsz, -1)
        m[:, -1] = 1; m = m.view(-1, 1)
        xmask *= m

        return xmeta,xmask

    def get_meta_fused_output(
        self, 
        input_ids:Optional[torch.Tensor]=None, 
        attention_mask:Optional[torch.Tensor]=None, 
        **kwargs
    ):
        data_h = self.get_output(input_ids, attention_mask)[0]

        meta_inputs = self._get_meta_inputs(self.aug_meta_prefix, **kwargs)
        for m in meta_inputs.values():
            valid_idx = torch.where(m['data2ptr'] > 0)[0]
            if len(valid_idx):
                bsz = len(valid_idx)
                meta_input_ids, meta_attention_mask = self.resize_meta(m['input_ids'], m['attention_mask'], 
                                                                       m['data2ptr'][valid_idx])
                meta_h = self.get_output(meta_input_ids, meta_attention_mask)[0]

                meta_h,meta_attention_mask = meta_h.view(bsz, -1, self.config.dim), meta_attention_mask.view(bsz, -1)

                data_h[valid_idx] += self.fuser(data_h[valid_idx], attention_mask[valid_idx], 
                                                meta_h, meta_attention_mask)[0]

        data_h = self.ln(data_h)
        return (data_h,)

    def get_meta_fused_genrep(
        self, 
        input_ids:Optional[torch.Tensor]=None, 
        attention_mask:Optional[torch.Tensor]=None, 
        **kwargs
    ):    
        o = self.get_meta_fused_output(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        )
        torch.cuda.empty_cache()

        rep = self.dr_transform(o[0])
        rep = self.dr_layer_norm(rep)
        rep = self.dr_projector(rep)
        rep = F.normalize(Pooling.mean_pooling(rep, attention_mask), dim=1)

        logits = self.vocab_transform(o[0])
        logits = self.activation(logits)
        logits = self.vocab_layer_norm(logits)
        logits = self.vocab_projector(logits)
        return o,logits,rep


### Example

In [ ]:
torch.manual_seed(100)

model = DBT018.from_pretrained('distilbert-base-uncased', ig_tok=0, tn_targ=10_000, tn_meta=10_000, 
                               margin=0.3, tau=0.1, n_negatives=5, apply_softmax=True, lw=0.01,
                               aug_meta_prefix='hlk', init_drh=True)

Some weights of DBT018 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight', 'fuser.k.bias', 'fuser.k.weight', 'fuser.o.bias', 'fuser.o.weight', 'fuser.q.bias', 'fuser.q.weight', 'fuser.v.bias', 'fuser.v.weight', 'ln.bias', 'ln.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr', 
    'pcat2lbl2data_idx', 'pcat2lbl2data_data2ptr', 'cat2lbl2data_idx', 'cat2lbl2data_input_ids', 
    'cat2lbl2data_attention_mask', 'cat2lbl2data_data2ptr',
    
    'phlk2data_idx', 'phlk2data_data2ptr', 'hlk2data_idx', 'hlk2data_input_ids', 'hlk2data_attention_mask', 
    'hlk2data_data2ptr'
])

In [ ]:
o = model(**b)

In [ ]:
o.loss

tensor(0.1434, grad_fn=<AddBackward0>)

## DBT019

In [ ]:
#| export
class DBT019(DBT018):
    
    @delegates(DBT018.__init__)
    def __init__(self, config, m_lw:Optional[Union[List,float]]=0.2, pred_meta_prefix:Optional[List]=None, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        store_attr('m_lw,pred_meta_prefix')
        
    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        data_o, data_logits, data_repr = self.get_meta_fused_genrep(data_input_ids, data_attention_mask, **kwargs)
        
        loss = lm_loss = dr_loss = lbl2data_repr = None
        if lbl2data_input_ids is not None:
            lbl2data_o, lbl2data_repr = self.get_representation(lbl2data_input_ids, lbl2data_attention_mask)
            lm_loss = self.gen_lfn(data_logits, lbl2data_input_ids, lbl2data_data2ptr, **kwargs)
            dr_loss = self.rep_lfn(data_repr, lbl2data_repr, lbl2data_data2ptr, lbl2data_idx, 
                                   plbl2data_data2ptr, plbl2data_idx, **kwargs)
            loss = dr_loss + self.lw*lm_loss
            
            meta_inputs = self._get_meta_inputs(self.pred_meta_prefix, **kwargs)
            if isinstance(self.m_lw, float):
                meta_lw = self.m_lw/len(meta_inputs) if len(meta_inputs) else None
                meta_lw = [meta_lw]*len(meta_inputs)
            else:
                if len(self.m_lw) != len(meta_inputs): raise ValueError(f'length of `m_lw` should be equal to number of metadata.')
                meta_lw = self.m_lw
            
            for m,m_lw in zip(meta_inputs.values(),meta_lw):
                valid_idx = torch.where(m['data2ptr'])[0]
                if len(valid_idx) > 0:
                    o, rep = self.get_representation(m['input_ids'], m['attention_mask'])
                    m_lml = self.gen_lfn(data_logits[valid_idx], m['input_ids'], m['data2ptr'][valid_idx], **kwargs)
                    m_drl = self.rep_lfn(data_repr[valid_idx], rep, m['data2ptr'][valid_idx], m['idx'], 
                                         m['pdata2ptr'][valid_idx], m['pidx'], **kwargs)
                    loss += m_lw * (m_drl + self.lw*m_lml)
            
        if not return_dict:
            o = (data_logits,data_repr,lbl2data_repr) + data_o[2:]
            return ((loss,lm_loss,dr_loss) + o) if loss is not None else o
        
        return XCModelOutput(
            loss=loss,
            lm_loss=lm_loss,
            dr_loss=dr_loss,
            logits=data_logits,
            data_repr=data_repr,
            lbl2data_repr=lbl2data_repr,
            data_hidden_states=data_o[0],
        )
    

### Example

In [ ]:
model = DBT019.from_pretrained('distilbert-base-uncased', ig_tok=0, tn_targ=10_000, tn_meta=10_000, 
                               margin=0.3, tau=0.1, n_negatives=5, apply_softmax=True, lw=0.01,
                               aug_meta_prefix='hlk', pred_meta_prefix='cat', m_lw=0.1, init_drh=True)

Some weights of DBT019 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight', 'fuser.k.bias', 'fuser.k.weight', 'fuser.o.bias', 'fuser.o.weight', 'fuser.q.bias', 'fuser.q.weight', 'fuser.v.bias', 'fuser.v.weight', 'ln.bias', 'ln.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr', 
    'pcat2lbl2data_idx', 'pcat2lbl2data_data2ptr', 'cat2lbl2data_idx', 'cat2lbl2data_input_ids', 
    'cat2lbl2data_attention_mask', 'cat2lbl2data_data2ptr',
    
    'phlk2data_idx', 'phlk2data_data2ptr', 'hlk2data_idx', 'hlk2data_input_ids', 'hlk2data_attention_mask', 
    'hlk2data_data2ptr'
])

In [ ]:
o = model(**b)

In [ ]:
o.loss

tensor(0.1621, grad_fn=<AddBackward0>)

## DBT020

In [ ]:
#| export
class DBT020(DBT018):
    
    @delegates(DBT018.__init__)
    def __init__(self, config, m_lw:Optional[Union[List,float]]=0.2, pred_meta_prefix:Optional[List]=None, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        store_attr('m_lw,pred_meta_prefix')
        
    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):  
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        data_o, data_logits, data_repr = self.get_meta_fused_genrep(data_input_ids, data_attention_mask, **kwargs)
        
        loss = lm_loss = dr_loss = lbl2data_repr = None
        if lbl2data_input_ids is not None:
            lbl2data_o, lbl2data_logits, lbl2data_repr = self.get_genrep(lbl2data_input_ids, lbl2data_attention_mask)
            lm_loss = self.gen_lfn(data_logits, lbl2data_input_ids, lbl2data_data2ptr, **kwargs)
            dr_loss = self.rep_lfn(data_repr, lbl2data_repr, lbl2data_data2ptr, lbl2data_idx, 
                                   plbl2data_data2ptr, plbl2data_idx, **kwargs)
            loss = dr_loss + self.lw*lm_loss
            
            meta_inputs = self._get_meta_inputs(self.pred_meta_prefix, **kwargs)
            if isinstance(self.m_lw, float):
                meta_lw = self.m_lw/len(meta_inputs) if len(meta_inputs) else None
                meta_lw = [meta_lw]*len(meta_inputs)
            else:
                if len(self.m_lw) != len(meta_inputs): raise ValueError(f'length of `m_lw` should be equal to number of metadata.')
                meta_lw = self.m_lw
            
            for m,m_lw in zip(meta_inputs.values(),meta_lw):
                if 'lbl2data2ptr' in m:
                    valid_idx = torch.where(m['lbl2data2ptr'])[0]
                    if len(valid_idx) > 0:
                        o, rep = self.get_representation(m['input_ids'], m['attention_mask'])
                        m_lml = self.gen_lfn(lbl2data_logits[valid_idx], m['input_ids'], m['lbl2data2ptr'][valid_idx], **kwargs)
                        m_drl = self.rep_lfn(lbl2data_repr[valid_idx], rep, m['lbl2data2ptr'][valid_idx], m['idx'], 
                                             m['plbl2data2ptr'][valid_idx], m['pidx'], **kwargs)
                        loss += m_lw * (m_drl + self.lw* m_lml)
                        
                elif 'data2ptr' in m:
                    valid_idx = torch.where(m['data2ptr'])[0]
                    if len(valid_idx) > 0:
                        o, rep = self.get_representation(m['input_ids'], m['attention_mask'])
                        m_lml = self.gen_lfn(data_logits[valid_idx], m['input_ids'], m['data2ptr'][valid_idx], **kwargs)
                        m_drl = self.rep_lfn(data_repr[valid_idx], rep, m['data2ptr'][valid_idx], m['idx'], 
                                             m['pdata2ptr'][valid_idx], m['pidx'], **kwargs)
                        loss += m_lw * (m_drl + self.lw*m_lml)       
                
                else: raise ValueError('Invalid metadata input arguments.')
            
        if not return_dict:
            o = (data_logits,data_repr,lbl2data_repr) + data_o[2:]
            return ((loss,lm_loss,dr_loss) + o) if loss is not None else o
        
        return XCModelOutput(
            loss=loss,
            lm_loss=lm_loss,
            dr_loss=dr_loss,
            logits=data_logits,
            data_repr=data_repr,
            lbl2data_repr=lbl2data_repr,
            data_hidden_states=data_o[0],
        )
    

### Example

In [ ]:
model = DBT020.from_pretrained('distilbert-base-uncased', ig_tok=0, tn_targ=10_000, tn_meta=10_000, 
                               margin=0.3, tau=0.1, n_negatives=5, apply_softmax=True, lw=0.01,
                               aug_meta_prefix='hlk', pred_meta_prefix='cat', m_lw=0.1, init_drh=True)

Some weights of DBT020 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight', 'fuser.k.bias', 'fuser.k.weight', 'fuser.o.bias', 'fuser.o.weight', 'fuser.q.bias', 'fuser.q.weight', 'fuser.v.bias', 'fuser.v.weight', 'ln.bias', 'ln.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
b = prepare_batch(model, batch, m_args=[
    'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_input_ids', 'cat2data_attention_mask', 
    'cat2data_data2ptr', 
    'pcat2lbl2data_idx', 'pcat2lbl2data_data2ptr', 'cat2lbl2data_idx', 'cat2lbl2data_input_ids', 
    'cat2lbl2data_attention_mask', 'cat2lbl2data_data2ptr',
    
    'phlk2data_idx', 'phlk2data_data2ptr', 'hlk2data_idx', 'hlk2data_input_ids', 'hlk2data_attention_mask', 
    'hlk2data_data2ptr'
])

In [ ]:
o = model(**b)

In [ ]:
o.loss

tensor(0.1624, grad_fn=<AddBackward0>)